Purpose of this notebook is to demonstrate dynamic programming as a method for policy approximation.

Aim is for you to understand
- concept of what we need to define a Markov Decision Process (state transitons, reward transitions)
- how we use the Bellman equation to bootstrap our value function approximation

- how dynamic programming can learn value functions without taking actions]
- how dynamic programming needs the environment model to work

http://bozskyfilip.blogspot.co.uk/2009/04/temporal-differential-learning-policy.html

In [2]:
import numpy as np

In [4]:
state_space = np.array(range(0,7))
terminal_states = [0,6]
action_space = np.array(['left', 'right'])
reward = [ 1 if s == 6 else 0 for s in state_space]

In [20]:
#  loading state transition probabilities from csvs
state_transitions = np.array[1, ]
state_transitions


array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [25]:
for i, state in enumerate(state_space[:-2]):
    state_transitions[0][i] = 1
    state_transitions[1][i+1] = 1
state_transitions    

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  1.,  0.]])

In [4]:
# the state transition probabilities is a lookup table
# note that rows = actions, columns = states
    
#        's1' 's2' 's3' 's4' 's5'   
# 'left'
# 'right'
# 'up'
# 'down

In [5]:
# here are the probabilities for state s1
# the probability of action 'right' taking us to 's2' is 1
# the probability of action 'up' taking us to 's1' is 1
# i.e. P(s'|s,a)

state_transitions['s1']

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

In [6]:
#  we can now do our reward transition probabilities
reward_functions = {state:np.full((len(state_space)),-1) for state in state_space}

#  these work in a similar way to our state transitions, except they are not conditioned upon actions
#  only conditions upon state, next_state
#  i.e R(s, s')
#  this is OK for our problem - a more formal problem would have R(s, a, s')

#  reward of all state->next_state transitons is -1
reward_functions['s4']

array([-1, -1, -1, -1, -1])

In [7]:
#  now we manually fill in the rewards for our special states
reward_functions['s4'][4] = 10
reward_functions['s4']

array([-1, -1, -1, -1, 10])

In [8]:
#  's5' is a special absorbing state with no rewards and transition prob of 1 to itself
reward_functions['s5'] = np.full((len(state_space)),0)
reward_functions['s5']

array([0, 0, 0, 0, 0])

In [9]:
#  we need a policy to approximate - lets use a random policy

def random_policy(state, action_space):
    """
    A random policy
    
    Args
        state        : str  : the name of the current state
        action_space : list : a list of all possible actions
        
    Returns
        action       : str  : the name of the selected action
        p_dist       : list : a probability distribution over the action space
    """
    action = np.random.choice(action_space)
    p_dist = np.full(len(action_space),  1/len(action_space))
    assert np.sum(p_dist) == 1
    return action, p_dist

In [10]:
class Dynamic_Programmer(object):
    """
    Policy approximator based on dynamic programming
    
    Args
        state_space  : list : a list of all possible states
        action_space : list : a list of all possible actions
        state_transitions : array : an array of the environment state transiton probabilities
                                    dimensions = [actions, states]
        reward_functions : array : an array of the rewards.  conditioned only on s, s' 
                                   dimensions = [state, state]
        policy : function : a function that maps state to action & a probability distribution over actions  
        discount : float : the discount factor (gamma)
        verbose : int : controls printing
        
    Methods
        policy transitions : creates the a probability distribution over states
                             conditioned on both environment & policy
    """
    def __init__(self, 
                 state_space, 
                 action_space, 
                 state_transitions, 
                 reward_functions,
                 policy,
                 discount,
                 verbose=1):    
        
        self.state_space = state_space
        self.action_space = action_space
        self.state_transitions = state_transitions
        self.reward_functions = reward_functions
        self.policy = policy
        self.discount = discount
        self.verbose = verbose
    
        self.v_table = np.full(len(state_space), 0.0)

    def policy_transitions(self, state):
        """
        For a given state, returns a distribution over
        next states (not over actions!)
        
        conditioned on both the environment (self.state_transitions)
        and the policy (act_p_dist)
        
        Args
            state : str : the current state
            
        Returns
            all_probs : array : probability distribution over states
        """
        state_idx = np.argwhere(state==self.state_space).flatten()
        #  our policy gives us a distribution over actions
        _, action_p_dist = self.policy(state, self.action_space)
        
        #  we can now use our perfect environment model to 
        #  develop the state transition probabilities
        
        for next_state in self.state_space:    
            all_probs = np.full(len(state_space),0)

            for act_idx, act_prob in enumerate(action_p_dist):   
                
                state_probs = self.state_transitions[state][act_idx].flatten()
                probs = act_prob * state_probs
                all_probs = np.add(all_probs, probs)

        print('policy transitions are {} for state {}'.format(all_probs, state))
        assert np.sum(all_probs) == 1        
        return all_probs
    
    def backup_state(self, state, probabilities): 
        """
        Backs up a given state & probability distribution
        
        Args
            state : str : the current state
            probabilities : array : probability distribution over states
            
        Returns
            self.v_table : array : the value function lookup table
        """
        rewards = self.reward_functions[state]
        discounted_values = self.discount * self.v_table  # note the recursive use of v.table!
        bellman_equations = np.add(rewards, discounted_values)
        
        print('probabilities {}'.format(probabilities))
        print('bellman equations {}'.format(bellman_equations))
        
        state_value = np.sum(np.multiply(probabilities, bellman_equations))
        print('state value of state {} is {}'.format(state, state_value))
        
        state_idx = np.argwhere(state==self.state_space).flatten()
        self.v_table[state_idx] = state_value
        
        if self.verbose == 1:
            print('backing up state {}'.format(state))
            print('probabilities are {}'.format(probabilities))
            print('rewards are {}'.format(rewards))
            print('discounted_values are {}'.format(discounted_values))  
            print('bellman equations are {}'.format(bellman_equations))
            print('state value is {}'.format(state_value))
            print('value function table {}'.format(self.v_table))
        return self.v_table
    
    def backup_all_states(self):
        """
        Runs a backup over the entire state space
        
        Returns
            v_table : array : the value function lookup table
        """
        for state in self.state_space:
            probabilities = self.policy_transitions(state)
            self.v_table = self.backup_state(state, probabilities)
        return self.v_table  
            

In [11]:
random_policy_approx = Dynamic_Programmer(state_space, 
                        action_space, 
                        state_transitions,
                        reward_functions,
                        random_policy,
                        discount=0.9,
                        verbose=0)
BACKUPS = 100
for backup in range(BACKUPS):
    value_function = random_policy_approx.backup_all_states()
    print('value function is {}'.format(value_function))

policy transitions are [ 0.5   0.25  0.25  0.    0.  ] for state s1
probabilities [ 0.5   0.25  0.25  0.    0.  ]
bellman equations [-1. -1. -1. -1. -1.]
state value of state s1 is -1.0
policy transitions are [ 0.25  0.5   0.    0.25  0.  ] for state s2
probabilities [ 0.25  0.5   0.    0.25  0.  ]
bellman equations [-1.9 -1.  -1.  -1.  -1. ]
state value of state s2 is -1.225
policy transitions are [ 0.25  0.    0.25  0.5   0.  ] for state s3
probabilities [ 0.25  0.    0.25  0.5   0.  ]
bellman equations [-1.9    -2.1025 -1.     -1.     -1.    ]
state value of state s3 is -1.225
policy transitions are [ 0.    0.25  0.25  0.25  0.25] for state s4
probabilities [ 0.    0.25  0.25  0.25  0.25]
bellman equations [ -1.9     -2.1025  -2.1025  -1.      10.    ]
state value of state s4 is 1.19875
policy transitions are [ 0.  0.  0.  0.  1.] for state s5
probabilities [ 0.  0.  0.  0.  1.]
bellman equations [-0.9      -1.1025   -1.1025    1.078875  0.      ]
state value of state s5 is 0.0
valu

probabilities [ 0.25  0.5   0.    0.25  0.  ]
bellman equations [-4.4546392  -3.73436692 -2.72773811 -0.26166265 -1.        ]
state value of state s2 is -3.0462589212830515
policy transitions are [ 0.25  0.    0.25  0.5   0.  ] for state s3
probabilities [ 0.25  0.    0.25  0.5   0.  ]
bellman equations [-4.4546392  -3.74163303 -2.72773811 -0.26166265 -1.        ]
state value of state s3 is -1.9264256529218564
policy transitions are [ 0.    0.25  0.25  0.25  0.25] for state s4
probabilities [ 0.    0.25  0.25  0.25  0.25]
bellman equations [ -4.4546392   -3.74163303  -2.73378309  -0.26166265  10.        ]
state value of state s4 is 0.8157303087208159
policy transitions are [ 0.  0.  0.  0.  1.] for state s5
probabilities [ 0.  0.  0.  0.  1.]
bellman equations [-3.4546392  -2.74163303 -1.73378309  0.73415728  0.        ]
state value of state s5 is 0.0
value function is [-3.838488   -3.04625892 -1.92642565  0.81573031  0.        ]
policy transitions are [ 0.5   0.25  0.25  0.    0.  ] f

bellman equations [-3.48763433 -2.76914031 -1.75666764  0.71833275  0.        ]
state value of state s5 is 0.0
value function is [-3.87514925 -3.07682257 -1.95185293  0.7981475   0.        ]
policy transitions are [ 0.5   0.25  0.25  0.    0.  ] for state s1
probabilities [ 0.5   0.25  0.25  0.    0.  ]
bellman equations [-4.48763433 -3.76914031 -2.75666764 -0.28166725 -1.        ]
state value of state s1 is -3.875269151276742
policy transitions are [ 0.25  0.5   0.    0.25  0.  ] for state s2
probabilities [ 0.25  0.5   0.    0.25  0.  ]
bellman equations [-4.48774224 -3.76914031 -2.75666764 -0.28166725 -1.        ]
state value of state s2 is -3.076922527147071
policy transitions are [ 0.25  0.    0.25  0.5   0.  ] for state s3
probabilities [ 0.25  0.    0.25  0.5   0.  ]
bellman equations [-4.48774224 -3.76923027 -2.75666764 -0.28166725 -1.        ]
state value of state s3 is -1.9519360927648433
policy transitions are [ 0.    0.25  0.25  0.25  0.25] for state s4
probabilities [ 0.  

state value of state s1 is -3.8757188290977096
policy transitions are [ 0.25  0.5   0.    0.25  0.  ] for state s2
probabilities [ 0.25  0.5   0.    0.25  0.  ]
bellman equations [-4.48814695 -3.76956544 -2.75702133 -0.28191182 -1.        ]
state value of state s2 is -3.077297413062963
policy transitions are [ 0.25  0.    0.25  0.5   0.  ] for state s3
probabilities [ 0.25  0.    0.25  0.5   0.  ]
bellman equations [-4.48814695 -3.76956767 -2.75702133 -0.28191182 -1.        ]
state value of state s3 is -1.9522479775624153
policy transitions are [ 0.    0.25  0.25  0.25  0.25] for state s4
probabilities [ 0.    0.25  0.25  0.25  0.25]
bellman equations [ -4.48814695  -3.76956767  -2.75702318  -0.28191182  10.        ]
state value of state s4 is 0.7978743323564581
policy transitions are [ 0.  0.  0.  0.  1.] for state s5
probabilities [ 0.  0.  0.  0.  1.]
bellman equations [-3.48814695 -2.76956767 -1.75702318  0.7180869   0.        ]
state value of state s5 is 0.0
value function is [-3.

policy transitions are [ 0.25  0.    0.25  0.5   0.  ] for state s3
probabilities [ 0.25  0.    0.25  0.5   0.  ]
bellman equations [-4.48815697 -3.76957603 -2.75703009 -0.28191788 -1.        ]
state value of state s3 is -1.952255702509536
policy transitions are [ 0.    0.25  0.25  0.25  0.25] for state s4
probabilities [ 0.    0.25  0.25  0.25  0.25]
bellman equations [ -4.48815697  -3.76957603  -2.75703013  -0.28191788  10.        ]
state value of state s4 is 0.7978689906059167
policy transitions are [ 0.  0.  0.  0.  1.] for state s5
probabilities [ 0.  0.  0.  0.  1.]
bellman equations [-3.48815697 -2.76957603 -1.75703013  0.71808209  0.        ]
state value of state s5 is 0.0
value function is [-3.87572997 -3.0773067  -1.9522557   0.79786899  0.        ]
policy transitions are [ 0.5   0.25  0.25  0.    0.  ] for state s1
probabilities [ 0.5   0.25  0.25  0.    0.  ]
bellman equations [-4.48815697 -3.76957603 -2.75703013 -0.28191791 -1.        ]
state value of state s1 is -3.875730

policy transitions are [ 0.    0.25  0.25  0.25  0.25] for state s4
probabilities [ 0.    0.25  0.25  0.25  0.25]
bellman equations [ -4.48815722  -3.76957624  -2.75703031  -0.28191803  10.        ]
state value of state s4 is 0.7978688562710488
policy transitions are [ 0.  0.  0.  0.  1.] for state s5
probabilities [ 0.  0.  0.  0.  1.]
bellman equations [-3.48815722 -2.76957624 -1.75703031  0.71808197  0.        ]
state value of state s5 is 0.0
value function is [-3.87573025 -3.07730693 -1.9522559   0.79786886  0.        ]
policy transitions are [ 0.5   0.25  0.25  0.    0.  ] for state s1
probabilities [ 0.5   0.25  0.25  0.    0.  ]
bellman equations [-4.48815722 -3.76957624 -2.75703031 -0.28191803 -1.        ]
state value of state s1 is -3.8757302476541935
policy transitions are [ 0.25  0.5   0.    0.25  0.  ] for state s2
probabilities [ 0.25  0.5   0.    0.25  0.  ]
bellman equations [-4.48815722 -3.76957624 -2.75703031 -0.28191803 -1.        ]
state value of state s2 is -3.07730

In [12]:
print('value function is {}'.format(value_function))

value function is [-3.87573025 -3.07730693 -1.9522559   0.79786886  0.        ]


In [13]:
#  we can now try to evaluate an optimal policy
#  for this simple MDP we can hard code the optimal policy

def optimal_policy(state, action_space):
    
    optimal_action = {'s1' : 'right',
                      's2' : 'down',
                      's3' : 'right',
                      's4' : 'right',
                      's5' : 'right'}

    action = optimal_action[state]

    p_distribution = np.full(len(action_space), 0)
    act_idx = np.argwhere(action == action_space)
    p_distribution[act_idx] = 1
    return action, p_distribution

In [14]:
optimal_policy_approx = Dynamic_Programmer(state_space, 
                        action_space, 
                        state_transitions,
                        reward_functions,
                        optimal_policy,
                        discount=0.9,
                        verbose=0)

BACKUPS = 10
for backup in range(BACKUPS):
    optimal_value_function = optimal_policy_approx.backup_all_states()
    print('value function is {}'.format(optimal_value_function))

policy transitions are [ 0.  1.  0.  0.  0.] for state s1
probabilities [ 0.  1.  0.  0.  0.]
bellman equations [-1. -1. -1. -1. -1.]
state value of state s1 is -1.0
policy transitions are [ 0.  0.  0.  1.  0.] for state s2
probabilities [ 0.  0.  0.  1.  0.]
bellman equations [-1.9 -1.  -1.  -1.  -1. ]
state value of state s2 is -1.0
policy transitions are [ 0.  0.  0.  1.  0.] for state s3
probabilities [ 0.  0.  0.  1.  0.]
bellman equations [-1.9 -1.9 -1.  -1.  -1. ]
state value of state s3 is -1.0
policy transitions are [ 0.  0.  0.  0.  1.] for state s4
probabilities [ 0.  0.  0.  0.  1.]
bellman equations [ -1.9  -1.9  -1.9  -1.   10. ]
state value of state s4 is 10.0
policy transitions are [ 0.  0.  0.  0.  1.] for state s5
probabilities [ 0.  0.  0.  0.  1.]
bellman equations [-0.9 -0.9 -0.9  9.   0. ]
state value of state s5 is 0.0
value function is [ -1.  -1.  -1.  10.   0.]
policy transitions are [ 0.  1.  0.  0.  0.] for state s1
probabilities [ 0.  1.  0.  0.  0.]
bellman